In [1]:
import sys
sys.path.append('../../src')

import pickle
from tuneable_counterfactuals_explainer.explainer import Explainer

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from tqdm import tqdm
import random

from alibi.explainers import CounterfactualProto

/Users/joshanparmar/Development/TuneableCounterfactuals/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

In [3]:
model = pickle.load(open('model/model.pkl', 'rb'))
training_data = pickle.load(open('model/training_data.pkl', 'rb'))
model[0].feature_names_in_ = None

In [4]:
x_train = training_data[training_data.columns[:-1]]

In [5]:
with open('results.pkl', 'rb') as f:
    old_results = pickle.load(f)

In [6]:
to_eval = list(old_results.keys())

In [7]:
def predict_fn(x): 
    prediction_array = np.array(x)
    if len(prediction_array.shape) == 1:
        prediction_array = np.array([prediction_array])
        
    return model.predict_proba(prediction_array)

cf = CounterfactualProto(
    predict_fn,
    shape = (1,) + x_train.shape[1:], # shape of input data
    # feature_range = (np.quantile(x_train.values.astype(float), 0.05, axis=0).reshape(1,-1), np.quantile(x_train.values.astype(float), 0.95, axis=0).reshape(1,-1)), # feature range for the perturbed counterfactual instance
    # lam_init = 1e-3, # initial lambda value
)

cf.fit(x_train.values.astype(float))

CounterfactualProto(meta={
  'name': 'CounterfactualProto',
  'type': ['blackbox', 'tensorflow', 'keras'],
  'explanations': ['local'],
  'params': {
              'shape': (1, 784),
              'kappa': 0.0,
              'beta': 0.1,
              'feature_range': (-10000000000.0, 10000000000.0),
              'gamma': 0.0,
              'theta': 0.0,
              'cat_vars': None,
              'ohe': False,
              'use_kdtree': False,
              'learning_rate_init': 0.01,
              'max_iterations': 1000,
              'c_init': 10.0,
              'c_steps': 10,
              'eps': (0.001, 0.001),
              'clip': (-1000.0, 1000.0),
              'update_num_grad': 1,
              'write_dir': None,
              'is_model': False,
              'is_ae': False,
              'is_enc': False,
              'enc_or_kdtree': False,
              'is_cat': False,
              'trustscore_kwargs': None,
              'd_type': 'abdm',
              'w': None,


In [8]:
results = {}
for id in tqdm(to_eval):
    results[id] = (cf.explain(np.array(x_train.iloc[id]).reshape(1, -1)))

100%|██████████| 25/25 [2:12:46<00:00, 318.65s/it]  


In [9]:
with open('results_prototype.pkl', 'wb') as f:   
    pickle.dump(results, f)